In [ ]:
# Grocery Sales Database EDA (Working Title)
# Presented by Michael, Eduardo, Terkula, Alexander, and Lu

In [ ]:
# Database can be found at https://www.kaggle.com/datasets/andrexibiza/grocery-sales-dataset

In [ ]:
# Questions
# What is the best-selling food product?
#     ... in the Northeast?
#     ... in the South?
#     ... in the Midwest?
#     ... in the Southwest?
#     ... in the Northwest?
#     ... outside the contiguous US states?
# What is the best-selling non-food product?
#     ... in the Northeast?
#     ... in the South?
#     ... in the Midwest?
#     ... in the Southwest?
#     ... in the Northwest?
#     ... outside the contiguous US states?
# What is the average cart value per customer?
#     ... can also be broken down per region
# What % of customers are repeat customers?
# What region had the most repeat customers?
# Which was the most popular seasonal product?
# What is the average store sales volume?
# How many stores exceeded average sales volume?
# When is customer volume highest?
#     ... per day?
#     ... per week?
#     ... per month?
#     ... per quarter?
# When were sales highest?
#      ... per day?
#      ... per week?
#      ... per month?
#      ... per quarter?
# Which flavor ice cream was most popular?




In [ ]:
# Data visualizations
# 1.
# 2.
# 3.
# 4.
# 5.
# 6.
# 7.
# 8.
# (more as needed)testing

In [ ]:
# Who's Doing What?
# Michael:
# Eduardo:
# Ter:
# Alexander:
# Lu: